In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import Necessary libraries 

In [99]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,Input,BatchNormalization,SimpleRNN
from keras.layers import Layer
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.python.keras import metrics
tf. executing_eagerly()

True

# Define MAP Metric

In [278]:
def apk(actual, predicted, k=3):
    
    actual = list(actual)
    predicted = list(predicted)
    
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
            
    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

# Read the Data

In [62]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/test.csv')

label = train[train.challenge_sequence > 10][['user_id','challenge']]
label.rename(columns={'challenge':'label'},inplace=True)

temp = train[train.challenge_sequence <= 10]
df = temp.groupby('user_id')["challenge"].agg(lambda x: ' '.join(x)).reset_index()

df = df.merge(label)

In [63]:
df

,user_id,challenge,label
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI24958
1,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23667
2,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23691
3,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI24915
4,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI25727
...,...,...,...
208591,113837,CI23880 CI23933 CI26052 CI24866 CI25049 CI2567...,CI23990
208592,113837,CI23880 CI23933 CI26052 CI24866 CI25049 CI2567...,CI25674
208593,113839,CI24915 CI24931 CI24954 CI24968 CI24957 CI2572...,CI28610
208594,113839,CI24915 CI24931 CI24954 CI24968 CI24957 CI2572...,CI26388


In [282]:
df_train, df_validation = df.iloc[:200001].reset_index(drop=True),df.iloc[200001:].reset_index(drop=True)

challenges= pd.read_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/RecommendationSystems/challenge_data.csv")

In [283]:
df_train

,user_id,challenge,label,brand_id_encoded
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI24958,1480
1,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23667,189
2,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23691,213
3,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI24915,1437
4,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI25727,2249
...,...,...,...,...
199996,109337,CI27778 CI27809 CI27811 CI27782 CI27779 CI2778...,CI27794,4316
199997,109337,CI27778 CI27809 CI27811 CI27782 CI27779 CI2778...,CI27797,4319
199998,109340,CI24876 CI24902 CI24908 CI24907 CI24960 CI2369...,CI25015,1537
199999,109340,CI24876 CI24902 CI24908 CI24907 CI24960 CI2369...,CI25111,1633


# Encoding Challenges

In [284]:
encoder = LabelEncoder()
encoder.fit(challenges['challenge'].astype(str))
df_train['brand_id_encoded'] = encoder.transform(df_train.label)
df_validation['brand_id_encoded'] = encoder.transform(df_validation.label)
df["brand_id_encoded"] = encoder.transform(df.label)

In [285]:
challenges.shape

(5606, 9)

# Tokenizing Challenges

In [286]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['challenge'])

TotalWords = len(tokenizer.word_index)
MAX_SEQUENCE_LENGTH = 10
N_CATEGORIES = challenges.shape[0]

sequences_train = tokenizer.texts_to_sequences(df_train['challenge'])
sequences_validation = tokenizer.texts_to_sequences(df_validation['challenge'])

# Padding sequences (not necessary)

In [287]:
#just to make it in to numpy array
x_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
x_validation = pad_sequences(sequences_validation, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

y_train = df_train['brand_id_encoded'].values
y_validation= df_validation['brand_id_encoded'].values

In [ ]:
#--------------------For whole Data-------------------------
tokenizer.fit_on_texts(df['challenge'])
sequences_whole_train = tokenizer.texts_to_sequences(df['challenge'])
whole_xtrain = pad_sequences(sequences_whole_train, maxlen=10, padding='post', truncating='post')
whole_ytrain = df['brand_id_encoded'].values
#------------------------------------------------------------

In [276]:
x_train[:10]

array([[  3,   1,   2,   4,   6,   8,  17,  11,  14,  37],
       [  3,   1,   2,   4,   6,   8,  17,  11,  14,  37],
       [  3,   1,   2,   4,   6,   8,  17,  11,  14,  37],
       [  4,   1,   6,  17,   7,   3,  19,  76,  36,   8],
       [  4,   1,   6,  17,   7,   3,  19,  76,  36,   8],
       [  4,   1,   6,  17,   7,   3,  19,  76,  36,   8],
       [186, 243, 213, 219, 223, 250, 280, 202, 162, 184],
       [186, 243, 213, 219, 223, 250, 280, 202, 162, 184],
       [186, 243, 213, 219, 223, 250, 280, 202, 162, 184],
       [  1,  14,   2,   6,   4,   5,  17,   3,  29, 105]], dtype=int32)

In [277]:
y_train[:10]

array([1480,  189,  213, 1437, 2249, 2573, 2685, 2688, 2689, 1657])

In [244]:
#inputX
display(x_train.shape)
#inputY
y_train.shape

((200000, 10), (200000,))

In [245]:
y_train

array([1480,  189,  213, ..., 4319, 1537, 1633])

# Testing different layers with Train and Val

### Using Attention

In [262]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [336]:
def get_model(path='',lr=0.002):
        adam = Adam(lr=lr,beta_1=0.5)
        inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
        x = Embedding(TotalWords+1,256)(inp)
        x = BatchNormalization()(x)
        x = Bidirectional(LSTM(256,  return_sequences=True, recurrent_dropout=0.1))(x)
        # x = Bidirectional(LSTM(32, dropout=0.1, return_sequences=True, recurrent_dropout=0.1))(x)
        # x = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.1))(x)
        # x = SimpleRNN(128,dropout=0.1,recurrent_dropout=0.1, return_sequences=True)(x)
        x = attention()(x)
        x = Dropout(0.1)(x)
        x = Dense(N_CATEGORIES, activation="softmax")(x)
        model = Model(inputs=inp, outputs=x)
        if path != '':
            model.load_weights(path)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
        return model

In [337]:
model = get_model()

path = 'best_model_weights2'
es_callback = EarlyStopping(monitor="val_loss", patience=5)
mc_callback = ModelCheckpoint(f'{path}.hdf5', monitor='val_loss',
                              verbose=0, save_best_only=True, save_weights_only=True,
                              mode='auto', period=1)
callbacks = [es_callback,mc_callback]

In [338]:
model.fit(x_train,
              y_train,
              epochs=20,
              batch_size=4100,
              validation_data=(x_validation, y_validation),
              callbacks = callbacks
              )

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 200001 samples, validate on 8595 samples
Epoch 1/20
200001/200001 [==============================] - 9s 47us/step - loss: 5.8962 - accuracy: 0.0379 - val_loss: 6.2910 - val_accuracy: 0.0212
Epoch 2/20
200001/200001 [==============================] - 9s 43us/step - loss: 4.7035 - accuracy: 0.0685 - val_loss: 5.8073 - val_accuracy: 0.0344
Epoch 3/20
200001/200001 [==============================] - 9s 43us/step - loss: 4.2839 - accuracy: 0.0946 - val_loss: 5.3700 - val_accuracy: 0.0531
Epoch 4/20
200001/200001 [==============================] - 9s 43us/step - loss: 3.9574 - accuracy: 0.1183 - val_loss: 5.1843 - val_accuracy: 0.0663
Epoch 5/20
200001/200001 [==============================] - 9s 44us/step - loss: 3.7001 - accuracy: 0.1354 - val_loss: 4.9469 - val_accuracy: 0.0810
Epoch 6/20
200001/200001 [==============================] - 9s 44us/step - loss: 3.5001 - accuracy: 0.1494 - val_loss: 4.8675 - val_accuracy: 0.0905
Epoch 7/20
200001/200001 [==============================

In [ ]:
#val-loss
# 4.4514    
# 4.3398 - best

In [339]:
val_data_for_pred = x_validation[0::3]# getting only 1 sequence
val_result_for_pred = y_validation.reshape(-1,3)

In [340]:
val_data_for_pred

array([[  64,  280,    5, ...,  106,  138,   12],
       [ 396,   31,   29, ..., 1682,  772, 2117],
       [ 396, 1232,    7, ..., 2324, 1328, 1735],
       ...,
       [  60,  135,   32, ...,  221,  276,    9],
       [  44,    6,   48, ...,   12,  148,  637],
       [  14,  135,  225, ..., 3294,  198,  137]], dtype=int32)

In [341]:
val_result_for_pred

array([[ 292, 1817, 1597],
       [1438, 2572,  236],
       [ 931, 1362, 2569],
       ...,
       [ 358,   32, 3143],
       [1414,  512, 2196],
       [5132, 2910, 5152]])

In [342]:
pred = model.predict(val_data_for_pred,batch_size=4096)
pred = pred.argsort(axis=1)[:,-3:][:,::-1]

In [343]:
pred.shape,val_data_for_pred.shape,val_result_for_pred.shape

((2865, 3), (2865, 10), (2865, 3))

In [344]:
mapk(val_result_for_pred,pred)

0.28181113050223

In [ ]:
#0.2818 on Validation --- 0.271 in public LeaderBoard

# Making Final Model

In [264]:
def get_model(path='',lr=0.002):

  adam = Adam(lr=lr,beta_1=0.5)
  # meta_inp = Input(shape=)

  inp = Input(shape=(MAX_SEQUENCE_LENGTH,),name='nlp_input')
  x = Embedding(TotalWords+1,256)(inp)
  x = BatchNormalization()(x)
  x = Bidirectional(LSTM(256,  return_sequences=True, recurrent_dropout=0.1))(x)

  x = Dropout(0.1)(x)
  x = Dense(N_CATEGORIES, activation="softmax")(x)
  model = Model(inputs=inp, outputs=x)
  if path != '':
      model.load_weights(path)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=["accuracy"])
  return model

In [265]:
model = get_model()

path = 'best_model_weights2'
es_callback = EarlyStopping(monitor="loss", patience=5)
mc_callback = ModelCheckpoint(f'{path}.hdf5', monitor='accuracy',
                              verbose=0, save_best_only=True, save_weights_only=True,
                              mode='auto', period=1)
callbacks = [es_callback,mc_callback]

In [273]:
model.fit(whole_xtrain,
              whole_ytrain,
              epochs=50,
              batch_size=4100,
              # validation_data=(x_validation, y_validation),
              callbacks = callbacks
              )

Epoch 1/50
208596/208596 [==============================] - 9s 42us/step - loss: 1.9911 - accuracy: 0.2308
Epoch 2/50
208596/208596 [==============================] - 9s 42us/step - loss: 1.9836 - accuracy: 0.2324
Epoch 3/50
208596/208596 [==============================] - 9s 43us/step - loss: 1.9754 - accuracy: 0.2344
Epoch 4/50
208596/208596 [==============================] - 9s 43us/step - loss: 1.9708 - accuracy: 0.2351
Epoch 5/50
208596/208596 [==============================] - 9s 43us/step - loss: 1.9675 - accuracy: 0.2347
Epoch 6/50
208596/208596 [==============================] - 9s 44us/step - loss: 1.9625 - accuracy: 0.2339
Epoch 7/50
208596/208596 [==============================] - 9s 44us/step - loss: 1.9572 - accuracy: 0.2359
Epoch 8/50
208596/208596 [==============================] - 9s 45us/step - loss: 1.9528 - accuracy: 0.2346
Epoch 9/50
208596/208596 [==============================] - 9s 45us/step - loss: 1.9481 - accuracy: 0.2356
Epoch 10/50
208596/208596 [==========

In [140]:
# accuracy--> Public MAP
# 0.2023 - 0.250
# 0.2242 - 0.258
# 0.2331 - 0.270
# 0.2411 - 0.271

# Predicting on Test

In [259]:
model = get_model(f'{path}.hdf5')

In [268]:
#just to make it in to numpy array
def padding(text):
    return pad_sequences(tokenizer.texts_to_sequences(text),
                         maxlen=MAX_SEQUENCE_LENGTH,
                         padding='post',
                         truncating='post')

In [269]:
temp = test[test.challenge_sequence <= 10]
test_text = temp.groupby('user_id')['challenge'].agg(lambda x: ' '.join(x)).reset_index()
x_test = padding(test_text.challenge)

In [345]:
%%time
pred = model.predict(x_test,batch_size=4096)
pred = pred.argsort(axis=1)[:,-3:][:,::-1]

CPU times: user 14.5 s, sys: 3.01 s, total: 17.5 s
Wall time: 17.6 s


In [307]:
x_test.shape,pred.shape

((39732, 10), (39732, 3))

In [310]:
x_test[:3],pred[:3]

(array([[  1,   6,   2,  14,   3,   4,   5,   8,  98,   7],
        [  4,   1,   2,   6,  17,   3,   8,  14,   5,  76],
        [395, 689, 569, 644, 544, 718, 695, 568, 495, 249]], dtype=int32),
 array([[1049, 1056, 1579],
        [ 370, 1052,  709],
        [3471, 3477, 3472]]))

In [346]:
df_list = []
for i in range(3):
    test_11 = test_text[['user_id']]
    test_11['user_sequence'] = test_11.user_id.astype(str) + '_'+str(i+11)
    test_11['challenge'] = encoder.inverse_transform(pred[:,i])
    df_list.append(test_11[['user_sequence','challenge']])
pd.concat(df_list).to_csv('Final.csv',index=False)